In [ ]:
import pandas as pd
import numpy as np
import opendatasets as od
import cloudpickle as cpickle

In [ ]:
# Set display options for columns and width
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # No limit on display width
pd.set_option('display.max_colwidth', None) # Show full width of each column

In [ ]:
od.download("https://www.kaggle.com/datasets/mathurinache/game-of-thrones-data?select=script-bag-of-words.json")

In [ ]:
df = pd.read_json("game-of-thrones-data/script-bag-of-words.json").set_index('episodeAlt')
df.head()

In [ ]:
df.iloc[-1]['text']

In [ ]:
dialogue = {}
for index,row in df.iterrows():
    for item in row['text']:
        if item['name'] in dialogue:
            dialogue[item['name']] = dialogue[item['name']] + item['text']
        else:
            dialogue[item['name']] = item['text'] + " "

In [ ]:
len(dialogue)

In [ ]:
new_df = pd.DataFrame(columns=["Character name", "Words", "Word Count"])
new_df['Character name'] = dialogue.keys()
new_df['Words'] = dialogue.values()
new_df["Word Count"] = new_df['Words'].apply(lambda x:len(x.split()))

In [ ]:
new_df = new_df.sort_values(ascending=False,by="Word Count")

In [ ]:
new_df.iloc[91,:]

In [ ]:
new_df = new_df[(new_df['Word Count'] >= 500)]

In [ ]:
new_df.shape

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words="english")

In [ ]:
embeddings = cv.fit_transform(new_df['Words']).toarray()

In [ ]:
embeddings.shape

In [ ]:
embeddings = embeddings.astype('float64')

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE(n_components=2,verbose=1,random_state=123)
z = tsne.fit_transform(embeddings)

In [ ]:
z.shape

In [ ]:
new_df['x'] = z.T[0]
new_df['y'] = z.T[1]
new_df

In [ ]:
import plotly.express as px
fig = px.scatter(new_df.head(25), x = "x", y="y", color="Character name")
fig.show()